1. Start from the rigid body formulation, using carden angle (mentioned in the paper), show pro/con versus using quaternion
2. Show the formulation of body-joint and how it used to model the "Youbot robot" 
3. Study structure of Modia 3d based on ModiaLang and ModiaBase
4. Study the solution sequence outlined in the paper andou the index reduction of DAE
5. Study the solution sequence of ModiaLang Tool kit

In [20]:
module Pendulum1
# import Modia3D
using Modia
@usingModiaPlot 
Modia.usePlotPackage("PyPlot")
Pendulum = Model(
    world     = Object3D(feature=Scene()),
    body      = Object3D(feature=Solid(massProperties=MassProperties(mass=1.0))),
    bodyFrame = Object3D(parent=:body, translation=[-0.5, 0.0, 0.0]),
    rev       = Revolute(obj1=:world, obj2=:bodyFrame)
)

# Pendulum = Pendulum | Map( bodyFrame = Object3D(parent=:body, translation=[-0.3, 0.0, 0.0]))

pendulum = @instantiateModel(buildModia3D(Pendulum), unitless=true)
simulate!(pendulum, stopTime=3.0u"s", merge = Map(bodyFrame = Object3D(parent=:body, translation=[-0.3, 0.0, 0.0])))
time=get_result(pendulum,"time")
angle=get_result(pendulum,"rev.phi")
# simulate!(pendulum, stopTime=3.0)

# @usingModiaPlot    ENV["MODIA_PLOT"] = "PyPlot" # use the plot package defined by ENV["MODIA_PLOT"]
# ModiaPlot_PyPlot.plot(pendulum, "rev.phi")
# plot(pendulum, "rev.phi")
plot(ustrip(time),ustrip(angle))
end

using ModiaPlot_PyPlot



Instantiating model Main.Pendulum1.buildModia3D(Pendulum)


LoadError: MethodError: no method matching addPlot(::Float64, ::Vector{Float64}, ::Bool, ::Bool, ::Nothing, ::String, ::Bool, ::Int64, ::Bool)
[0mClosest candidates are:
[0m  addPlot([91m::Tuple[39m, ::Any, ::Bool, ::Bool, ::Any, ::AbstractString, ::Bool, ::Integer, ::Bool) at C:\Users\clair\.julia\packages\ModiaPlot_PyPlot\n6Rca\src\ModiaPlot_PyPlot.jl:109
[0m  addPlot([91m::AbstractString[39m, ::Any...) at C:\Users\clair\.julia\packages\ModiaPlot_PyPlot\n6Rca\src\ModiaPlot_PyPlot.jl:138
[0m  addPlot([91m::Symbol[39m, ::Any...) at C:\Users\clair\.julia\packages\ModiaPlot_PyPlot\n6Rca\src\ModiaPlot_PyPlot.jl:139

In [34]:
module Pendulum2

using Modia
# import Modia3D

Pendulum = Model(
    world = Object3D(feature=Scene(animationFile="Pendulum2.json")),
#     obj1  = Object3D(feature=Solid(shape=Cone(axis=1, diameter=1.0, length=1.0, topDiameter=0.0),
#                 solidMaterial="Steel", visualMaterial=VisualMaterial(color="Blue"))),
    
        obj1  = Object3D(feature=Solid(shape=Box(lengthX=1.0, lengthY=0.2, lengthZ=0.2),
                solidMaterial="Steel", visualMaterial=VisualMaterial(color="Blue"))),
    
#     obj1  = Object3D(feature=Solid(shape=Beam(axis=1, length=1.0, width=0.2, thickness=0.2),
#                 solidMaterial="Steel", visualMaterial=VisualMaterial(color="Blue"))),
    obj2  = Object3D(parent=:obj1, feature=Visual(shape=Cylinder(diameter=0.1, length=0.21),
                visualMaterial=VisualMaterial(color="Red")), translation=[-0.5, 0.0, 0.0]),
    rev   = Revolute(obj1=:world, obj2=:obj2)
)

pendulum = @instantiateModel(buildModia3D(Pendulum), unitless=true)
simulate!(pendulum, stopTime=3.0)

@usingModiaPlot
plot(pendulum, "rev.phi")

end


Instantiating model Main.Pendulum2.buildModia3D(Pendulum)


Export animation to Pendulum2.json ... done.
using ModiaPlot_PyPlot


Main.Pendulum2

In [35]:
module Pendulum3

using Modia

# Modia equation-based models
include("$(Modia.modelsPath)/AllModels.jl")

Pendulum = Model(
    world = Object3D(feature=Scene(animationFile="Pendulum3.json")),
#     obj1  = Object3D(feature=Solid(shape=Beam(axis=1, length=1.0, width=0.2, thickness=0.2),
#                 solidMaterial="Steel", visualMaterial=VisualMaterial(color="Blue"))),
    obj1  = Object3D(feature=Solid(shape=Box(lengthX=1.0, lengthY=0.2, lengthZ=0.2),
                solidMaterial="Steel", visualMaterial=VisualMaterial(color="Blue"))),
    
    obj2  = Object3D(parent=:obj1, feature=Visual(shape=Cylinder(diameter=0.1, length=0.21),
                visualMaterial=VisualMaterial(color="Red")), translation=[-0.5, 0.0, 0.0]),
    rev   = RevoluteWithFlange(obj1=:world, obj2=:obj2),

    damper  = Damper | Map(d=100.0),
    fixed   = Fixed,
    connect = :[(damper.flange_b, rev.flange),
                (damper.flange_a, fixed.flange)]
)

pendulum = @instantiateModel(buildModia3D(Pendulum), unitless=true)
simulate!(pendulum, stopTime=3.0)

@usingModiaPlot
plot(pendulum, "rev.phi")

end


Instantiating model Main.Pendulum3.buildModia3D(Pendulum)
Export animation to Pendulum3.json ... done.
using ModiaPlot_PyPlot


Main.Pendulum3

# Function of RevoluteWithFlange: 
How the RevoluteWithFlange function has been written

In [ ]:
RevoluteWithFlange(; obj1, obj2, axis=3, phi=Var(init=0.0), w=Var(init=0.0),canCollide=true) 


= Model(;_constructor = Par(value =:(Modia3D.Composition.Revolute),_jointType = :RevoluteWithFlange),
obj1 = Par(value = obj1),
obj2 = Par(value = obj2),
axis = Par(value = axis),
canCollide = Par(value = canCollide),
flange = Flange, # defined in Rotational
phi = phi,
w = w,
equations = :[
phi = flange.phi
w = der(phi)])

In [70]:
import Modia3D
include("$(Modia3D.path)/test/Tutorial/Pendulum3.jl")


Instantiating model Main.Pendulum3.buildModia3D(Pendulum)


Export animation to Pendulum3.json ... done.
using ModiaPlot_PyPlot


Main.Pendulum3

In [71]:
import Modia3D
include("$(Modia3D.path)/test/Tutorial/BouncingSphere.jl")

LoadError: SystemError: opening file "C:\\Users\\clair\\.julia\\packages\\Modia3D\\cpwrJ\\test\\Tutorial\\BouncingSphere.jl": No such file or directory

In [65]:
import Modia3D
include("$(Modia3D.path)/test/Tutorial/Pendulum1.jl")


Instantiating model Main.Pendulum1.buildModia3D(Pendulum)


using ModiaPlot_PyPlot


Main.Pendulum1

In [76]:
import Modia3D
include("$(Modia3D.path)/test/Tutorial/Pendulum2.jl")


Instantiating model Main.Pendulum2.buildModia3D(Pendulum)


Export animation to Pendulum2.json ... done.
using ModiaPlot_PyPlot


Main.Pendulum2

In [75]:
using Modia
Modia.usePlotPackage("PyPlot")
# @usingModiaPlot  # Use plot package defined with 
                 # ENV["MODIA_PLOT"] or Modia.usePlotPackage(..)
# Modia.usePlotPackage("PyPlot")
Pendulum = Model(
   L = 0.8u"m",
   m = 1.0u"kg",
   d = 0.5u"N*m*s/rad",
   g = 9.81u"m/s^2",
   phi = Var(init = 1.57*u"rad"),
   w   = Var(init = 0u"rad/s"),
   equations = :[
          w = der(phi)
        0.0 = m*L^2*der(w) + d*w + m*g*L*sin(phi)
          r = [L*cos(phi), -L*sin(phi)]
   ]
)

pendulum1 = @instantiateModel(Pendulum)
simulate!(pendulum1, Tsit5(), stopTime = 10.0u"s", log=true)
ModiaPlot_PyPlot.plot(pendulum1, [("phi", "w"); "r"], figure = 1)


Instantiating model Main.Pendulum
... Simulate model Pendulum
      Initialization at time = 0.0 s
      Termination of Pendulum at time = 10.0 s
        cpuTime         = 0.114 s
        allocated       = 4.13 MiB
        algorithm       = Tsit5
        FloatType       = Float64
        interval        = 0.02 s
        tolerance       = 1.0e-6 (relative tolerance)
        nStates         = 2
        nResults        = 501
        nGetDerivatives = 1476 (total number of getDerivatives! calls)
        nf              = 973 (number of getDerivatives! calls from integrator)
        nZeroCrossings  = 0 (number of getDerivatives! calls for zero crossing detection)
        nJac            = 0 (number of Jacobian computations)
        nAcceptedSteps  = 162
        nRejectedSteps  = 0
        nErrTestFails   = 0
        nTimeEvents     = 0
        nStateEvents    = 0
        nRestartEvents  = 0


In [26]:
using Modia
@usingModiaPlot #ENV["MODIA_PLOT"] = "PyPlot"  # Use plot package defined with 
Modia.usePlotPackage("PyPlot")           # ENV["MODIA_PLOT"] or Modia.usePlotPackage(..)

Pendulum = Model(
   L = 0.8u"m",
   m = 1.0u"kg",
   d = 0.5u"N*m*s/rad",
   g = 9.81u"m/s^2",
   phi = Var(init = 1.57*u"rad"),
   w   = Var(init = 0u"rad/s"),
   equations = :[
          w = der(phi)
        0.0 = m*L^2*der(w) + d*w + m*g*L*sin(phi)
          r = [L*cos(phi), -L*sin(phi)]
   ]
)
Pendulum = Pendulum | Map(L = 0.5u"m")
pendulum1 = @instantiateModel(Pendulum)
simulate!(pendulum1, Tsit5(), stopTime = 10.0u"s", log=true)
# ENV["MODIA_PLOT"] = "PyPlot"
# ModiaResult.activate("PyPlot")
ModiaPlot_PyPlot.plot(pendulum1, [("phi", "w"); "r"], figure = 1)

using ModiaPlot_PyPlot

Instantiating model Main.Pendulum
... Simulate model Pendulum
      Initialization at time = 0.0 s
      Termination of Pendulum at time = 10.0 s
        cpuTime         = 0.0555 s
        allocated       = 4.11 MiB
        algorithm       = Tsit5
        FloatType       = Float64
        interval        = 0.02 s
        tolerance       = 1.0e-6 (relative tolerance)
        nStates         = 2
        nResults        = 501
        nGetDerivatives = 1416 (total number of getDerivatives! calls)
        nf              = 913 (number of getDerivatives! calls from integrator)
        nZeroCrossings  = 0 (number of getDerivatives! calls for zero crossing detection)
        nJac            = 0 (number of Jacobian computations)
        nAcceptedSteps  = 152
        nRejectedSteps  = 0
        nErrTestFails   = 0
        nTimeEvents     = 0
        nStateEvents    = 0
        nRestartEvents  = 0


***ModiaBase provides functions to support the transformation from a DAE to explicit ODE**